Первым делом надо сделать установку **ortools**. Сделать это достаточно просто, нужно просто раскомментировать строчку ниже и запустить ее. После ее отработки, скорее всего, понадобится перезапустить ноутбук. **Среда выполнения -> Перезапустить среду выполнения.(или Ctrl+M)**

In [257]:
# !pip install ortools

In [258]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


In [259]:
# Cоздаем класс Plan для более удобной работы с результатом отработки OR-Tools
class Plan:
  def __init__(self):
    self.path = []
    self.load = None
    self.output = {}
    self.times = []
    self.output['path'] = []
    

In [260]:
# В этой ячейке надо здать координаты расположения камеры
almtvsk_cams_coords = [(54.848387, 52.253091), #отправная точка автотранспорта, выбрана случайно)
                     (54.902532, 52.289729), # 0 - Белоглазова 131
                     (54.900249, 52.287286), # 1 - Белоглазова 151
                     (54.905891, 52.270927), # 3 - Гафиатуллина 29Б
                     (54.906383, 52.266427), # 4 - улица Гафиатуллина, 39
                     (54.905829, 52.264387), # 5 - улица Гафиатуллина, 45
                     (54.906518, 52.263822), # 6 - улица Гафиатуллина, 47
                     (54.906518, 52.263822), # 7 - улица Гафиатуллина, 47
                     (54.900078, 52.290205), # 10 - улица Ленина, 66
                     (54.898670, 52.285722), # 11 - улица Ленина, 90
                     (54.896071, 52.288390), # 12 - улица Шевченко, 80
                     (54.900777, 52.267460), # 14 - проспект Строителей, 20Б
                     (54.902294, 52.270047), # 15 - проспект Строителей, 20(тут фото с камеры обрезано, мб это не тот дом)
]

In [261]:
# В этой ячейке надо задать временные характеристики
import numpy as np
data = {}
time_matrix = [ # Время поездки между узлами
        [0, 6,   9,   8,   7,  3,    6,  2,  3,    2,   6,    6,   4],
        [6, 0,   8,   3,   2,  6,    8,  4,  8,    8,  13,    7,  5,],
        [9, 8,   0,  11,  10,  6,    3,  9,  5,    8,   4,   15,  14],
        [8, 3,  11,   0,   1,  7,   10,  6,  10,  10,  14,    6,   7],
        [7, 2,  10,   1,   0,  6,    9,  4,  8,    9,  13,    4,   6],
        [3, 6,   6,   7,   6,  0,    2,  3,  2,    2,   7,    9,   7],
        [6, 8,   3,  10,   9,  2,    0,  6,  2,    5,   4,   12,  10],
        [2, 4,   9,   6,   4,  3,    6,  0,  4,    4,   8,    5,   4],
        [3, 8,   5,  10,   8,  2,    2,  4,  0,    3,   4,    9,   8],
        [2, 8,   8,  10,   9,  2,    5,  4,  3,    0,   4,    6,   5],
        [6, 13,  4,  14,  13,  7,    4,  8,  4,    4,   0,   10,   9],
        [6, 7,  15,   6,  4,   9,   12,  5,  9,    6,  10,    0,   1],
        [4, 5,  14,   7,  6,   7,   10,  4,  8,    5,   9,    1,   0],
    ]

time_windows = [ # Время, когда мы можем заехать на конкретную точку
        (0, 100),  # depot
        (7, 12),  # 1
        (10, 15),  # 2
        (16, 18),  # 3
        (10, 13),  # 4
        (0, 5),  # 5
        (5, 10),  # 6
        (0, 4),  # 7
        (5, 10),  # 8
        (0, 3),  # 9
        (10, 16),  # 10
        (10, 15),  # 11
        (0, 5),  # 12
    ]

In [262]:
# Задаем кол-во(в у.е, по-нашему, тут задается оценочное значение в литрах или в кг) мусора на точке
# В MVP данный массив является результатом работы сетки
almtvsk_demands = [0,2,5,3,6,8,4,9,5,7,3,6,9]

In [263]:
# Задаем входные данные
almtvsk_data = {} # пустой словарик
almtvsk_data['num_vehicles'] = 4 # кол-во автомобилей в автопарке
almtvsk_data['depot'] = 0 # номер точки, из которой мы стартуем
almtvsk_data['demands'] = almtvsk_demands # кол-во мусора на точках
almtvsk_data['vehicle_capacities'] = [20, 20, 20, 20] # вместимость автомобилей(в тех же у.е., что и demands)
almtvsk_data['time_matrix'] = time_matrix # Время поездки между узлами
almtvsk_data['time_windows'] = time_windows # Время, когда мы можем заехать на конкретную точку
almtvsk_data # отображаем

{'demands': [0, 2, 5, 3, 6, 8, 4, 9, 5, 7, 3, 6, 9],
 'depot': 0,
 'num_vehicles': 4,
 'time_matrix': [[0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4],
  [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5],
  [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14],
  [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7],
  [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6],
  [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7],
  [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10],
  [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4],
  [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8],
  [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5],
  [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9],
  [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1],
  [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0]],
 'time_windows': [(0, 100),
  (7, 12),
  (10, 15),
  (16, 18),
  (10, 13),
  (0, 5),
  (5, 10),
  (0, 4),
  (5, 10),
  (0, 3),
  (10, 16),
  (10, 15),
  (0, 5)],
 'vehicle_capacities': [20, 20, 20, 20]}

In [264]:
# Функция отображения полученного решения
def print_solution(almtvsk_data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    # Display dropped nodes.
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))
    print(dropped_nodes)
    total_distance = 0
    total_load = 0
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    all_plans = []
    output_constants = ['lat', 'lon']
    for vehicle_id in range(almtvsk_data['num_vehicles']):
        plan = Plan()
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_load = 0
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            node_index = manager.IndexToNode(index)
            route_load += almtvsk_data['demands'][node_index]
            plan_output += ' {0} <Load({1}) || Time({2},{3})> -> '.format(node_index, route_load, solution.Min(time_var),
                solution.Max(time_var))
            plan.path.append((node_index, route_load))
            plan.output['path'].append({output_constants[i]: almtvsk_cams_coords[node_index][i] for i in (0,1)})
            previous_index = index
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += ' {0} <Load({1}) || Time({2},{3})>\n'.format(manager.IndexToNode(index),
                                                 route_load,
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Load of the route: {}\n'.format(route_load)
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        print(plan_output)
        plan.path.append((manager.IndexToNode(index), route_load))
        plan.output['path'].append({output_constants[i]: almtvsk_cams_coords[manager.IndexToNode(index)][i] for i in (0,1)})
        plan.load = route_load
        all_plans.append(plan)
        for i,res in enumerate(all_plans):
          all_plans[i].output['path'] = str(res.output['path']).replace('},',"};").replace(']',"").replace('[',"").replace(" ","")
        total_load += route_load
        total_time += solution.Min(time_var)
    print('Total load of all routes: {}'.format(total_load))
    print('Total time of all routes: {}min'.format(total_time))
    return all_plans

In [265]:
import numpy as np
# Вспомогательная функция построения матрицы расстояний по вектору координат
def form_distance_matrix(coordinates):
    X = np.array(coordinates)
    distance_matrix = np.zeros((len(coordinates), len(coordinates)))
    # distance_matrix = np.sqrt(np.sum((X[:,np.newaxis]-X)**2,axis=2)) # эвклидово растояние
    distance_matrix = np.sum(abs(X[:,np.newaxis]-X),axis=2) # Манхэтанская
    return distance_matrix

In [266]:
# Еще одна вспомогательная функция, отображает матрицу numpy в массив массивов
def np_to_list(arr):
  return [[arr[i,j] for j in range(len(arr[0]))] for i in range(len(arr))]

In [267]:
# Ячейчка, в которой решается оптимизационная задача. По сути, здесь происходят 
# необходимые вызовы из OR - Tools
def solver():
  def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return almtvsk_data['demands'][from_node]

  def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return almtvsk_data['distance_matrix'][from_node][to_node]

    # Create and register a transit callback.
  def time_callback(from_index, to_index):
    """Returns the travel time between the two nodes."""
    # Convert from routing variable Index to time matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return almtvsk_data['time_matrix'][from_node][to_node]

    # Instantiate the data problem.
  almtvsk_data['distance_matrix'] = np_to_list(form_distance_matrix(almtvsk_cams_coords))


    # Create the routing index manager.
  manager = pywrapcp.RoutingIndexManager(len(almtvsk_data['distance_matrix']),
                                           almtvsk_data['num_vehicles'], almtvsk_data['depot'])

    # Create Routing Model.
  routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.

  transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
  demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
  routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        almtvsk_data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

  transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
  time = 'Time'
  routing.AddDimension(
      transit_callback_index,
      30,  # allow waiting time
      30,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
  time_dimension = routing.GetDimensionOrDie(time)
  # Add time window constraints for each location except depot.
  for location_idx, time_window in enumerate(almtvsk_data['time_windows']):
      if location_idx == almtvsk_data['depot']:
          continue
      index = manager.NodeToIndex(location_idx)
      time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
  # Add time window constraints for each vehicle start node.
  depot_idx = almtvsk_data['depot']
  for vehicle_id in range(almtvsk_data['num_vehicles']):
      index = routing.Start(vehicle_id)
      time_dimension.CumulVar(index).SetRange(
          almtvsk_data['time_windows'][depot_idx][0],
          almtvsk_data['time_windows'][depot_idx][1])

    # Instantiate route start and end times to produce feasible times.
  for i in range(almtvsk_data['num_vehicles']):
      routing.AddVariableMinimizedByFinalizer(
          time_dimension.CumulVar(routing.Start(i)))
      routing.AddVariableMinimizedByFinalizer(
          time_dimension.CumulVar(routing.End(i)))



  # Allow to drop nodes.
  penalty = 1000
  for node in range(1, len(almtvsk_data['distance_matrix'])):
      routing.AddDisjunction([manager.NodeToIndex(node)], penalty)        
    
    # Setting first solution heuristic.
  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
  search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
  search_parameters.time_limit.FromSeconds(5)

    # Solve the problem.
  solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
  if solution:
      return print_solution(almtvsk_data, manager, routing, solution)

  else:
    print("No solution found")

Sample text

In [268]:
# Решаем задачу
almtvsk_result = solver()


Objective: 53
Dropped nodes:
Route for vehicle 0:
 0 <Load(0) || Time(0,0)> ->  8 <Load(5) || Time(5,5)> ->  6 <Load(9) || Time(7,7)> ->  2 <Load(14) || Time(10,10)> ->  10 <Load(17) || Time(14,14)> ->  0 <Load(17) || Time(20,20)>
Load of the route: 17
Time of the route: 20min

Route for vehicle 1:
 0 <Load(0) || Time(0,0)> ->  12 <Load(9) || Time(4,5)> ->  11 <Load(15) || Time(10,10)> ->  0 <Load(15) || Time(16,16)>
Load of the route: 15
Time of the route: 16min

Route for vehicle 2:
 0 <Load(0) || Time(0,0)> ->  7 <Load(9) || Time(2,4)> ->  1 <Load(11) || Time(7,11)> ->  4 <Load(17) || Time(10,13)> ->  3 <Load(20) || Time(16,16)> ->  0 <Load(20) || Time(24,24)>
Load of the route: 20
Time of the route: 24min

Route for vehicle 3:
 0 <Load(0) || Time(0,0)> ->  9 <Load(7) || Time(2,2)> ->  5 <Load(15) || Time(4,4)> ->  0 <Load(15) || Time(7,7)>
Load of the route: 15
Time of the route: 7min

Total load of all routes: 67
Total time of all routes: 67min


In [269]:
# Информация, которая передается в БД бота для дальнейшей обработки
try:
  for res in almtvsk_result:
      print(res.output['path'])
except TypeError:
    pass

{'lat':54.848387,'lon':52.253091};{'lat':54.900078,'lon':52.290205};{'lat':54.906518,'lon':52.263822};{'lat':54.900249,'lon':52.287286};{'lat':54.896071,'lon':52.28839};{'lat':54.848387,'lon':52.253091}
{'lat':54.848387,'lon':52.253091};{'lat':54.902294,'lon':52.270047};{'lat':54.900777,'lon':52.26746};{'lat':54.848387,'lon':52.253091}
{'lat':54.848387,'lon':52.253091};{'lat':54.906518,'lon':52.263822};{'lat':54.902532,'lon':52.289729};{'lat':54.906383,'lon':52.266427};{'lat':54.905891,'lon':52.270927};{'lat':54.848387,'lon':52.253091}
{'lat':54.848387,'lon':52.253091};{'lat':54.89867,'lon':52.285722};{'lat':54.905829,'lon':52.264387};{'lat':54.848387,'lon':52.253091}
